Code to reproduce running example of

B. Legat, R. M. Jungers, and P. A. Parrilo
[**Certifying unstability of Switched Systems using Sum of Squares Programming**](https://arxiv.org/abs/1710.01814)

In [ ]:
Pkg.clone("https://github.com/blegat/HybridSystems.jl")
Pkg.clone("https://github.com/blegat/SwitchOnSafety.jl")

In [27]:
include(Pkg.dir("SwitchOnSafety", "test", "solvers.jl"))
@assert !isempty(sdp_solvers)
solver = first(sdp_solvers)
include(Pkg.dir("HybridSystems", "examples", "PEDJ16s4.jl"))
using SwitchOnSafety

In [28]:
using Mosek
solver = MosekSolver(LOG=0);

## Example 3.6

In [9]:
using MultivariatePolynomials
function plot_lyap(p)
    d = mindegree(p)
    @assert d == maxdegree(p)
    @assert nvariables(p) == 2
    α = linspace(0., 2π, 1000)
    X = cos.(α)
    Y = sin.(α)
    λ = map(x -> p(variables(p) => x), zip(X, Y))
    λ = λ.^(1/d)
    plot(X ./ λ, Y ./ λ, aspect_ratio=:equal, label="", grid=false, border=nothing, ticks=nothing, linewidth=3)#, grid=false, label="")
    plot!(X, Y, label="", linestyle=:dash, linecolor=:black, linewidth=2)
end

plot_lyap (generic function with 1 method)

In [ ]:
gr()
for d in 1:6
    for v in 1:4
        plot_lyap(get(s.lyaps[d]).primal[v])
        savefig("PEDJ_d" * string(d) * "_v" * string(v) * ".png")
    end
end

## Example 4.3

In [26]:
using MultivariatePolynomials
using MultivariateMoments
d = 1
tol = 1e-4
lyap = getlyap(hs, d, solver=solver, tol=tol)
for s in states(hs)
    for t in out_transitions(hs, s)
        σ = symbol(hs, t)
        μ = SwitchOnSafety.measurefor(lyap.dual, hs, t)
        X = monomials(variables(μ), d)
        ν = matmeasure(μ, X)
        a = extractatoms(ν, tol)
        if !isnull(a)
            v = target(hs, t)
            x = get(a).support[1]
            nx = norm(x, 2)
            y = round.(x / nx, 3)
            println("μ_{$s$v$σ} = $nx * δ_{$y}")
            @show normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y)
        end
    end
end

μ_{131} = 1.090815554022926 * δ_{[0.917, 0.399]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.999626, -0.027334]
μ_{312} = 1.1435640854730555 * δ_{[0.874, 0.485]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.916725, 0.399518]
μ_{331} = 1.3211597229435221 * δ_{[0.757, -0.654]}
normalize(SwitchOnSafety.dynamicforσ(hs, σ) * y) = [0.421783, -0.906697]


## Example 4.10

In [3]:
function sequencelb(s, d, l)
    best = 0.
    for p_0 in (:Primal, :Random)
        for v_0 in states(s)
            seq = sosbuildsequence(s, d, p_0=p_0, v_0=v_0, l=l, niter=100)
            psw = findsmp(seq)
            @assert !isnull(psw)
            best = max(best, get(psw).growthrate)
        end
    end
    best
end

sequencelb (generic function with 1 method)

In [29]:
soslb = zeros(6)
sosub = zeros(6)
sosex = zeros(6)
seql1 = zeros(6)
seql2 = zeros(6)
seql3 = zeros(6)
seql4 = zeros(6)
seql5 = zeros(6);

In [ ]:
tol = 5e-4

for d in 1:6
    sosdata(hs).lb = 0.
    sosdata(hs).smp = nothing
    soslb[d], sosub[d] = soslyapb(hs, d, solver=solver, tol=tol, ranktols=[1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], disttols=[1e-5, 1e-4, 1e-3, 1e-2, 5e-2, 1e-1])
    if hassmp(hs)
        sosex[d] = getsmp(hs).growthrate
    end
    seql1[d] = sequencelb(hs, d, 1)
    seql2[d] = sequencelb(hs, d, 2)
    seql3[d] = sequencelb(hs, d, 3)
end

In [ ]:
using Plots
plotlyjs()

In [7]:
font = Plots.Font("sans-serif",12,:hcenter,:vcenter,0.0,RGB{FixedPointNumbers.Normed{UInt8, 8}}(0.0,0.0,0.0))
plot(1:6, xlabel = "d", sosub, label="SOS UB", legendfont=font, linewidth=3)
plot!(1:6, seql3, label="Algorithm 1, l=3", linewidth=3)
plot!(1:6, seql1, label="Algorithm 1, l=1", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")
plot!(1:6, sosex, label="Atom extraction", linewidth=3)
plot!(1:6, soslb, label="SOS LB", linewidth=3)

In [8]:
plot(2:5, xlabel = "d", xticks=2:5, sosub[2:5], label="", linewidth=3)
plot!(2:5, seql3[2:5], label="", linewidth=3)
plot!(2:5, seql1[2:5], label="", linewidth=3)
#plot!(1:6, seql2, label="Algorithm 1, l=2")